## DataCommunitySA Challenge

In [60]:
import os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [79]:
## loading COVID-19 data 
data = pd.read_csv('COVID-19/dati-regioni/dpc-covid19-ita-regioni-20200322.csv')

## Inspecting data
data

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-03-22 17:00:00,ITA,13,Abruzzo,42.351222,13.398438,218,49,267,272,539,45,15,33,587,3375
1,2020-03-22 17:00:00,ITA,17,Basilicata,40.639471,15.805148,13,10,23,58,81,15,0,0,81,643
2,2020-03-22 17:00:00,ITA,4,P.A. Bolzano,46.499335,11.356624,146,32,178,470,648,48,7,23,678,5718
3,2020-03-22 17:00:00,ITA,18,Calabria,38.905976,16.594402,77,17,94,166,260,35,5,8,273,3666
4,2020-03-22 17:00:00,ITA,15,Campania,40.839566,14.250850,243,99,342,524,866,73,41,29,936,4943
5,2020-03-22 17:00:00,ITA,8,Emilia Romagna,44.494367,11.341721,2429,269,2698,3692,6390,729,349,816,7555,28022
6,2020-03-22 17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,163,47,210,528,738,72,89,47,874,6761
7,2020-03-22 17:00:00,ITA,12,Lazio,41.892770,12.483667,671,79,750,522,1272,186,58,53,1383,17845
8,2020-03-22 17:00:00,ITA,7,Liguria,44.411493,8.932699,736,132,868,483,1351,192,143,171,1665,4995
9,2020-03-22 17:00:00,ITA,3,Lombardia,45.466794,9.190347,9439,1142,10581,7304,17885,515,5865,3456,27206,70598


In [67]:
## loading weather data from 20 file with taking the temp_avg and humidity_avg

regions = []
temps = []
humidity = []

path = "ITA_weather/*.csv"
for filename in glob.glob(path):
     regions.append(filename[12:-4])
     temps.append(pd.read_csv(filename)['T2M'].mean())
     humidity.append(pd.read_csv(filename)['RH2M'].mean())

ITA = pd.DataFrame()
ITA['Region'] = regions
ITA['temp_avg'] = temps
ITA['humidity_avg'] = humidity
ITA.head()

,Region,temp_avg,humidity_avg
0,Abruzzo,4.766026,80.275897
1,Basilicata,6.567308,79.182692
2,Calabria,11.668077,75.078590
3,Campania,11.454872,75.552692
4,EmiliaRomagna,6.103205,84.408846


## Data Cleaning
- renameing some columns.
- keeping the coulmns that I need.

In [68]:
data = data.rename(columns={'denominazione_regione':'Region', 'ricoverati_con_sintomi':'with_symptoms', 'terapia_intensiva':'intensive Care', 'totale_ospedalizzati':'total_hospitalised_patients', 'isolamento_domiciliare':'home_confinement', 'totale_attualmente_positivi':'current_positive_cases', 'dimessi_guariti':'Recovered', 'deceduti':'Deaths', 'totale_casi':'Confirmed', 'tamponi':'test_performed'})
c_data = data.groupby('Region').max().reset_index()[['Region', 'Confirmed', 'Recovered', 'Deaths']]
c_data.head()

,Region,Confirmed,Recovered,Deaths
0,Abruzzo,587,15,33
1,Basilicata,81,0,0
2,Calabria,273,5,8
3,Campania,936,41,29
4,Emilia Romagna,7555,349,816


In [69]:
## Merging the dataframes
m_data = pd.merge(c_data, ITA).sort_values(by='Confirmed', ascending=False)
m_data.head()

,Region,Confirmed,Recovered,Deaths,temp_avg,humidity_avg
6,Lombardia,27206,5865,3456,6.261282,76.708205
18,Veneto,5122,309,169,7.770641,80.317692
11,Piemonte,4420,10,283,3.250897,71.038205
7,Marche,2421,6,184,11.650256,71.933077
15,Toscana,2277,42,91,6.120641,86.047949


In [73]:
x = m_data[1:]

fig = px.scatter(x, y="temp_avg", x="Confirmed", title='Confirmed vs temp_avg')
fig.show()

fig = px.scatter(x, y="humidity_avg", x="Confirmed", title='Confirmed vs humidity_avg')
fig.show()

fig = px.bar(x, y="Confirmed", x="Region", title='Confirmed vs Region')
fig.show()

In [86]:


data_list = []

for r in c_data['Region']:
    data_list.append(data[data['Region'] == r])


    
    
print(data_list[0])

KeyError: 'Region'

## Resourses
- regional_data.csv: https://www.kaggle.com/bsridatta/covid-19-italy-updated-regularly#regional_data.csv
- 